In [1]:
import numpy as np
import pandas as pd

In [2]:
returns = pd.read_csv('analysis csv file/returns.csv')
sentiment_score_df = pd.read_csv('analysis csv file/sentiment_score_df.csv')

In [3]:
returns

,Ticker,Date,Return,Event
0,XLB,2021-09-14,-0.0110,Event 1
1,XLE,2021-09-14,-0.0140,Event 1
2,XLF,2021-09-14,-0.0134,Event 1
3,XLI,2021-09-14,-0.0120,Event 1
4,XLK,2021-09-14,-0.0010,Event 1
...,...,...,...,...
3746,XLRE,2023-01-20,0.0116,Event 10
3747,XLU,2023-01-20,0.0060,Event 10
3748,XLV,2023-01-20,0.0050,Event 10
3749,XLY,2023-01-20,0.0242,Event 10


In [4]:
announce_date = pd.DataFrame({'Date': ['2021-09-24', '2021-11-09', '2021-11-29', '2022-03-21', '2022-05-24', '2022-06-17', '2022-08-26', '2022-09-28', '2022-11-30', '2023-01-11']})
announce_date['Date'] = pd.to_datetime(announce_date['Date'])

In [5]:
window = 10
event_dates = ['2021-09-24', '2021-11-09', '2021-11-29', '2022-03-21', '2022-05-24', '2022-06-17', '2022-08-26', '2022-09-28', '2022-11-30', '2023-01-11']
event_dates = [pd.Timestamp(date) for date in event_dates]
returns['Date'] = pd.to_datetime(returns['Date'])

for event_date in event_dates:
    before_event = returns[returns['Date'] < event_date].tail(window)
    after_event = returns[returns['Date'] > event_date].head(window)

    mean_before = before_event['Return'].mean()
    std_before = before_event['Return'].std()
    mean_after = after_event['Return'].mean()
    std_after = after_event['Return'].std()

    return_mean = (mean_before-mean_after)/mean_before
    return_std = (std_before-std_after)/std_before
    announce_date.loc[announce_date['Date'] == event_date, 'Return_mean'] = return_mean
    announce_date.loc[announce_date['Date'] == event_date, 'Return_Standard_Deviation'] = return_std

sentiment_score_df['Date'] = pd.to_datetime(sentiment_score_df['Date'])
merged_df = pd.merge(announce_date, sentiment_score_df, on='Date')
print(merged_df)

corr_matrix = merged_df[['Return_mean', 'Return_Standard_Deviation', 'SentimentScore']].corr()
print(corr_matrix)


        Date  Return_mean  Return_Standard_Deviation  SentimentScore
0 2021-09-24     0.936924                  -0.298629               8
1 2021-11-09    -3.196928                  -0.136439              26
2 2021-11-29     0.069060                   0.400765              21
3 2022-03-21    -0.106188                   0.005310               2
4 2022-05-24     0.520890                  -0.200474              28
5 2022-06-17     1.815041                   0.313768               9
6 2022-08-26     1.290273                  -0.577637              18
7 2022-09-28    -3.113639                  -0.364481              12
8 2022-11-30     1.642191                   0.699407              28
                           Return_mean  Return_Standard_Deviation  \
Return_mean                   1.000000                   0.349404   
Return_Standard_Deviation     0.349404                   1.000000   
SentimentScore               -0.063444                   0.227383   

                           Sentim

In [6]:
import pandas as pd

# Assuming you have the returns, announce_date, and sentiment_score_df dataframes

window = 10
event_dates = ['2021-09-24', '2021-11-09', '2021-11-29', '2022-03-21', '2022-05-24', '2022-06-17', '2022-08-26', '2022-09-28', '2022-11-30', '2023-01-11']
event_dates = [pd.Timestamp(date) for date in event_dates]
returns['Date'] = pd.to_datetime(returns['Date'])

# Get unique tickers
tickers = returns['Ticker'].unique()

# Create an empty DataFrame to store the results
results = []

for ticker in tickers:
    ticker_returns = returns[returns['Ticker'] == ticker]
    for event_date in event_dates:
        before_event = ticker_returns[ticker_returns['Date'] < event_date].tail(window)
        after_event = ticker_returns[ticker_returns['Date'] > event_date].head(window)

        mean_before = before_event['Return'].mean()
        std_before = before_event['Return'].std()
        mean_after = after_event['Return'].mean()
        std_after = after_event['Return'].std()

        if mean_before != 0:
            return_mean = (mean_before - mean_after) / mean_before
        else:
            return_mean = None

        if std_before != 0:
            return_std = (std_before - std_after) / std_before
        else:
            return_std = None

        results.append({'Ticker': ticker, 'Date': event_date, 'Return_mean': return_mean, 'Return_std': return_std})

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Create a DataFrame with sentiment scores for each ticker
sentiment_score_df['Date'] = pd.to_datetime(sentiment_score_df['Date'])
sentiment_scores_tickers = pd.concat([sentiment_score_df.assign(Ticker=ticker) for ticker in tickers], ignore_index=True)

# Merge sentiment scores with the results DataFrame
merged_df = pd.merge(results_df, sentiment_scores_tickers, on=['Ticker', 'Date'])
print(merged_df)


   Ticker       Date  Return_mean  Return_std  SentimentScore
0     XLB 2021-09-24     1.055172    0.228157               8
1     XLB 2021-11-09     1.270270    0.368564              26
2     XLB 2021-11-29     1.314286   -1.130570              21
3     XLB 2022-03-21     0.230769    0.456828               2
4     XLB 2022-05-24    -0.320000   -0.044449              28
..    ...        ...          ...         ...             ...
94    XLC 2022-05-24    -2.015306    0.120532              28
95    XLC 2022-06-17     1.334322    0.162240               9
96    XLC 2022-08-26     1.562500    0.064757              18
97    XLC 2022-09-28     0.558089   -1.031141              12
98    XLC 2022-11-30    -2.878378   -0.194767              28

[99 rows x 5 columns]


In [7]:
# Group the merged DataFrame by Ticker and calculate the correlation matrix for each group
grouped = merged_df.groupby('Ticker')

# Initialize an empty list to store the correlation results
ticker_correlations = []

# Loop through each group (ticker) and calculate the correlation matrix
for ticker, group in grouped:
    corr_matrix = group[['Return_mean', 'Return_std', 'SentimentScore']].corr()
    corr_matrix['Ticker'] = ticker
    ticker_correlations.append(corr_matrix)

# Concatenate the list of DataFrames into a single DataFrame
ticker_correlations = pd.concat(ticker_correlations)

# Reset the index and filter out unnecessary columns
ticker_correlations.reset_index(inplace=True)
ticker_correlations = ticker_correlations[['Ticker', 'index', 'Return_mean', 'Return_std', 'SentimentScore']]

# Rename the columns for better readability
ticker_correlations.columns = ['Ticker', 'Metric', 'Return_mean_corr', 'Return_std_corr', 'SentimentScore_corr']


In [8]:
ticker_correlations

,Ticker,Metric,Return_mean_corr,Return_std_corr,SentimentScore_corr
0,XLB,Return_mean,1.000000,-0.250130,-0.411511
1,XLB,Return_std,-0.250130,1.000000,-0.160990
2,XLB,SentimentScore,-0.411511,-0.160990,1.000000
3,XLC,Return_mean,1.000000,-0.323311,-0.033343
4,XLC,Return_std,-0.323311,1.000000,-0.168661
5,XLC,SentimentScore,-0.033343,-0.168661,1.000000
6,XLE,Return_mean,1.000000,-0.015576,-0.451910
7,XLE,Return_std,-0.015576,1.000000,-0.392416
8,XLE,SentimentScore,-0.451910,-0.392416,1.000000
9,XLF,Return_mean,1.000000,-0.172528,0.099480


In [9]:
ticker_correlations.to_csv('analysis csv file/ticker_correlations.csv', index=False)